In [ ]:
from typing import List
import random


class Snake:
    def __init__(self, player_id):
        self.player_id = player_id
        self.body = []
        self.direction = (0, 0)
        self.score = 0
        self.food_eaten = 0

    def spawn(self):
        self.body = [(random.randint(5, 15), random.randint(5, 15))]
        self.direction = (1, 0)
        self.score = 0

    def move(self):

        head = self.body[0]
        new_head = (head[0] + self.direction[0], head[1] + self.direction[1])

        self.body.insert(0, new_head)
        if self.food_eaten > 0:
            self.food_eaten -= 1
        else:
            self.body.pop()



class Food:
    def __init__(self):
        self.position = (0, 0)
        self.spawn()

    def spawn(self):
        self.position = (random.randint(0, 19), random.randint(0, 19))



class MultiplayerSnakeGame:
    def __init__(self):
        self.width = 20
        self.height = 20
        self.snakes: List[Snake] = []
        self.foods: List[Food] = []
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]

    def add_snake(self, snake: Snake):
        self.snakes.append(snake)
        for x, y in snake.body:
            self.map[x][y] = snake.player_id

    def remove_snake(self, player_id: str):
        self.snakes = [snake for snake in self.snakes if snake.player_id != player_id]
        for x in range(self.width):
            for y in range(self.height):
                if self.map[x][y] == player_id:
                    self.map[x][y] = ""

    def add_food(self, food: Food):
        self.foods.append(food)
        self.map[food.position[0]][food.position[1]] = "food"

    def remove_food(self, position):
        self.foods = [food for food in self.foods if food.position != position]
        self.map[position[0]][position[1]] = ""

    def move_snakes(self):
        for snake in self.snakes:
            snake.move()

    def change_direction(self, player_id, direction):
        for snake in self.snakes:
            if snake.player_id == player_id:
                if (
                    snake.direction == (1, 0) and direction == (-1, 0)
                    or snake.direction == (-1, 0) and direction == (1, 0)
                    or snake.direction == (0, 1) and direction == (0, -1)
                    or snake.direction == (0, -1) and direction == (0, 1)
                ):
                    return
                snake.direction = direction

    def check_collisions(self):
        for snake in self.snakes:
            head = snake.body[0]
            if (
                head[0] < 0
                or head[0] >= self.width
                or head[1] < 0
                or head[1] >= self.height
            ):
                self.remove_snake(snake.player_id)
            elif (
                self.map[head[0]][head[1]] != "" and self.map[head[0]][head[1]] != "food"
            ):
                print("Collision")
                self.remove_snake(snake.player_id)
            elif head in [food.position for food in self.foods]:
                snake.score += 1
                snake.food_eaten += 1
                self.remove_food(head)

    def update_map(self):
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]
        for snake in self.snakes:
            for x, y in snake.body:
                self.map[x][y] = snake.player_id
        for food in self.foods:
            self.map[food.position[0]][food.position[1]] = "food"

    def spawn_food(self):
        if len(self.foods) >= 10:
            return
        new_food = Food()
        new_food.spawn()
        self.add_food(new_food)

    def spawn_snake(self, player_id):
        new_snake = Snake(player_id)
        new_snake.spawn()
        self.add_snake(new_snake)

    def update(self):
        self.move_snakes()
        self.check_collisions()
        self.update_map()
        self.spawn_food()


In [10]:
import time
from IPython.display import display, clear_output
from ipywidgets import Button, Label, GridspecLayout, VBox

game = MultiplayerSnakeGame()

players = {}
game_active = True
next_player_id = 0
FRAME_RATE = 0.5

# Create a function to display the game map using GridspecLayout
def display_2d_list(data):
    rows = len(data)
    cols = len(data[0]) if rows else 0
    grid = GridspecLayout(rows, cols)
    
    for i in range(rows):
        for j in range(cols):
            cell_label = Label(value=str(data[i][j]))
            cell_label.layout.border = '1px solid black'
            cell_label.layout.margin = '0px'  # Remove default margins
            cell_label.layout.padding = '0px'  # Remove default padding
            cell_label.layout.width = 'auto'  # Adjust width to fit content + border
            grid[i, j] = cell_label
    
    grid.layout.grid_gap = '0px'  # Eliminate gaps between grid cells
    return grid

# Initially display the game map
map_widget = display_2d_list(game.map)  # Assuming game.map is available and correctly initialized

# Create Button widgets for the controls
up_button = Button(description="Up")
down_button = Button(description="Down")
left_button = Button(description="Left")
right_button = Button(description="Right")

# Define the control functions
def move_up(b):
    game.change_direction((0, -1))

def move_down(b):
    game.change_direction((0, 1))

def move_left(b):
    game.change_direction((-1, 0))

def move_right(b):
    game.change_direction((1, 0))

# Attach the control functions to the Button widgets
up_button.on_click(move_up)
down_button.on_click(move_down)
left_button.on_click(move_left)
right_button.on_click(move_right)

# VBox to hold the game display and controls
game_display = VBox([map_widget, up_button, down_button, left_button, right_button])
display(game_display)

game.spawn_snake("player")
map_widget = display_2d_list(game.map)
# Record the start time
start_time = time.time()

while game_active:
    # Calculate the elapsed time
    elapsed_time = time.time() - start_time

    # If the elapsed time is greater than or equal to the frame rate, update the game
    if elapsed_time >= FRAME_RATE:
        game.update()
        # Update the display with the new state of the game map
        map_widget = display_2d_list(game.map)  # Re-create the grid with updated map data
        game_display.children = [map_widget, up_button, down_button, left_button, right_button]  # Update the VBox children

        # Reset the start time
        start_time = time.time()

KeyboardInterrupt: 

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time


def update_display_2d_list(initial_data, num_updates=5, delay=1):
    rows = len(initial_data)
    cols = len(initial_data[0]) if rows else 0
    
    # Create a grid layout
    grid = widgets.GridspecLayout(rows, cols)
    
    # Initialize the grid with the initial data
    for i in range(rows):
        for j in range(cols):
            grid[i, j] = widgets.Label(value=str(initial_data[i][j]))

    # VBox to hold the grid
    vbox = widgets.VBox([grid])
    
    # Display the VBox
    display(vbox)
    
    # Update loop
    for update in range(num_updates):
        # Simulate data update by modifying the 2D list
        for i in range(rows):
            for j in range(cols):
                new_value = str(initial_data[i][j]) + str(update)  # Example update
                grid[i, j].value = new_value  # Update the grid cell value
        time.sleep(delay)  # Delay to simulate time passing

# Example initial data
data = [
    ["A", "B"],
    ["C", "D"]
]

# Call the function to start the update display process
update_display_2d_list(data, num_updates=10, delay=1)



KeyboardInterrupt: 